```
Funções/métodos mínimos de entrega:
    
- calcula_prazo(dt_ini, dt_fim, feriados, convencao)
    - Retorna: prazo anualizado (float)
    - convencao: 'DU/252' ou 'DC/360'

- constroi_fluxo(dt_fim, frequencia)
    - Retorna: Lista de datas dos fluxos (list datetime.date)

- calcula_pu(VF, prazo_anual, taxa_anual)
    - Retorna pu (float)

- calcula_taxa_anual(PU, prazo_anual, valor_base=100)
    - Retorna: taxa_anual (float)

- calcula_pu_ntnf(dt_venc, dt_base, tir) *
    - Retorna: pu (float)
    - Imprime tabela com o cashflow (Data do fluxo, VF, DU, Fator de desconto, PU)
```

## Funções auxiliares, tipagem, e outros


In [2]:
import datetime as dt
from typing import Literal, get_args

import numpy as np
import pandas as pd

# Tipagem
Convencao = Literal[252, 360]
Frequencia = Literal["B","C","D","W","M","SM","BM","CBM","MS","SMS","BMS","CBMS","Q","BQ","QS","BQS","H","S","N",]

#DF de feriados
feriados = pd.read_csv("feriados/feriados_nacionais.xlsx - Plan1.csv",skiprows=list(range(936, 947)),usecols=[0],)
#Pegando somente as datas 
feriados_datas = pd.to_datetime(feriados["Data"], format="%m/%d/%Y").values.astype("datetime64[D]")


def fazTeste(valor_esperado, valor_real) -> bool:
    """Compara um valor esperado e um valor real
    valor_esperado: any
    valor_real: any

    retorno:
    True -> valor_esperado == valor_real
    False -> valor_esperado != valor_real
    """

    # Caso especial, se forem dois dataframes
    if isinstance(valor_esperado, pd.DataFrame) and isinstance(valor_real, pd.DataFrame):
        sucesso = len(valor_esperado.compare(valor_real)) == 0
    else:
        sucesso = valor_esperado == valor_real
        
    if sucesso:
        print("Teste sucedido!")
        return True

    print("Teste falhou!")
    print(f"Esperava: {valor_esperado}")
    print(f"Recebeu: {valor_real}")
    return False


def arredonda(num: float, casas: int = 2) -> float:
    pot10 = 10**casas
    return round(num * pot10) / pot10


def converte_str_em_data(datestr: str) -> dt.date:
    return pd.to_datetime(datestr).to_pydatetime().date()

## 1. Calcula Prazo


### 1.1 Função

In [3]:

def calcula_prazo(dt_ini: dt.date, dt_fim: dt.date, feriados: pd.DataFrame, convencao: Convencao) -> float:
    """
    Calcula o prazo de um produto de renda fixa.
    Recebe:
    dt_ini -> data inicial
    dt_fim -> data final
    feriados -> lista de feriados
    convencao -> convenção de prazo (252 ou 360)

    Retorna:
    um prazo float
    """
    if not isinstance(dt_ini, dt.date):
        # Tenta converter
        dt_ini = converte_str_em_data(dt_ini)
        if not isinstance(dt_ini, dt.date):
            raise TypeError(f"Data Inicio informada ({dt_ini}) não é do tipo datetime.date")

    if not isinstance(dt_fim, dt.date):
        # Tenta converter
        dt_fim = converte_str_em_data(dt_fim)
        if not isinstance(dt_fim, dt.date):
            raise TypeError(f"Data fim informada ({dt_fim}) não é do tipo datetime.date")

    if dt_fim < dt_ini:
        raise ValueError(f"Data fim informada ({dt_fim}) deve ser maior do que data inicio ({dt_ini})")
    convencao = int(convencao)

    if int(convencao) not in get_args(Convencao):
        raise TypeError(f"Convenção informada não é suportada pelo sistema")

    dt_ini = np.datetime64(dt_ini) + 1
    dt_fim = np.datetime64(dt_fim)  # Não subtrai 1 pois np.busday não conta o ultimo dia

    if convencao == 252:
        prazo = np.busday_count(
                                begindates=dt_ini,
                                enddates=dt_fim,
                                holidays=feriados,
                                weekmask="Mon Tue Wed Thu Fri",
                                )
        return prazo / 252
    
    elif convencao == 360:
        prazo = (dt_fim - dt_ini) / np.timedelta64(1, "D")
        return prazo / 360

### 1.2 Teste

In [4]:
# Teste
resultado_previsto = 13 / 252

dt_ini = dt.date(2022, 6, 10)
dt_fim = dt.date(2022, 7, 1)
resultado = calcula_prazo(dt_ini=dt_ini, dt_fim=dt_fim, convencao="252", feriados=feriados_datas)

fazTeste(resultado_previsto, resultado)

Teste sucedido!


True

## 2. Constrói Fluxo


### 2.1 Função

In [5]:
def construi_fluxo(dt_ini: dt.date, dt_fim: dt.date, frequencia: str):
    """Constroi um fluxo de datas a partir de uma data inicial,final e uma frequência
    Retorna um intervalo de datas
    """

    if np.datetime64(dt_fim) < np.datetime64(dt_ini):
        raise ValueError("Data de vencimento anterior a data atual")
    
    # Frequencia pode ser um multiplo. Retiramos os digitos e checamos se é um date offset alias
    i = 0
    while True:
        if not frequencia[i].isdigit():
            break
        i += 1
    if frequencia[i:] not in get_args(Frequencia):
        raise TypeError("Frequência recebida não é uma das frequências aceitas")

    date_interval = pd.date_range(start=np.datetime64(dt_ini), end=np.datetime64(dt_fim), freq=frequencia).values.astype("datetime64[D]")
    diff = date_interval[-1] - np.datetime64(dt_fim)
    return [(date - diff) for date in date_interval]

### 2.2 Teste

In [6]:
# Teste
resultado_previsto = [
                        dt.date(2022, 7, 1),
                        dt.date(2023, 1, 1),
                        dt.date(2023, 7, 1),
                        dt.date(2024, 1, 1),
                        dt.date(2024, 7, 1),
                        dt.date(2025, 1, 1),
                        dt.date(2025, 7, 1),
                        dt.date(2026, 1, 1),
                        dt.date(2026, 7, 1),
                        dt.date(2027, 1, 1),
                    ]

dt_ini = dt.date(2022, 6, 10)
dt_fim = dt.date(2027, 1, 1)
resultado = construi_fluxo(dt_ini, dt_fim, frequencia="6M")
fazTeste(resultado_previsto, resultado)

Teste sucedido!


True

## 3. Calcula PU


### 3.1 Função

In [8]:
def calcula_pu(VF: float, prazo_anual: float, taxa_anual: float) -> float:
    """Calcula o PU
    VF: float
    prazo_anual: float
    taxa_anual: float

    retorno:
    PU: float
    """
    return VF / ((1 + taxa_anual) ** (prazo_anual))

### 3.2 Teste

In [9]:
# Testa calcula_pu
VF = 48.81
prazo_anual = 13 / 252
taxa_anual = 0.1265
resultado_previsto = 48.51
resultado = arredonda(calcula_pu(VF, prazo_anual, taxa_anual), 2)
fazTeste(resultado_previsto, resultado)


Teste sucedido!


True

## 4. Calcula taxa anual


### 4.1 Função

In [10]:
def calcula_taxa_anual(PU: float, prazo_anual: float, valor_base: float = 100) -> float:
    """Calcula a taxa anual
    PU: float
    prazo_anual: float
    valor_base: int
    """

    # PU = VF / ((1 + taxa_anual) ** (prazo_anual))
    # ((1 + taxa_anual) ** (prazo_anual)) = VF / PU
    # 1 + taxa_anual = (VF / PU) ** (1/prazo_anual)

    return ((valor_base / PU) ** (1 / prazo_anual)) - 1

### 4.2 Teste

In [11]:
# Testa calcula_taxa_anual
VF = 48.81
prazo_anual = 13 / 252
resultado_previsto = 0.127
PU = 48.51
resultado = arredonda(calcula_taxa_anual(PU, prazo_anual, VF), 3)
fazTeste(resultado_previsto, resultado)


Teste sucedido!


True

## 5. Calcula Pu Ntn-F


### 5.1 Função

In [12]:
# Função
def calcula_pu_ntnf(dt_venc: dt.date, dt_base: dt.date, tir: float, valor_base: float = 1000):
    
    CONVENCAO_DATA = 252
    
    fluxo = sorted(construi_fluxo(dt_base, dt_venc, "6M"), reverse=True)

    ntnf = pd.DataFrame(fluxo, columns=["Data Fluxo"])
    ntnf["Juros"] = arredonda(valor_base * ((1 + 0.1) ** (6/12) - 1), 4)
    ntnf["Amortização"] = np.where(ntnf["Data Fluxo"] == np.datetime64(dt_venc), valor_base, 0)
    ntnf["VF"] = ntnf["Juros"] + ntnf["Amortização"]
    ntnf["Prazo DU"] = ntnf["Data Fluxo"].apply(lambda row: calcula_prazo(dt_base, row.date(), feriados_datas, CONVENCAO_DATA) * CONVENCAO_DATA)
    ntnf["FD"] = arredonda((1 + tir) ** (ntnf["Prazo DU"] / CONVENCAO_DATA), 9)
    ntnf["PU"] = arredonda(ntnf["VF"] / ntnf["FD"], 2)

    return ntnf

### 5.2 Teste

In [13]:
# Teste

# Dados tirados da planilha da Larissa
resultado_previsto = pd.DataFrame(
                                    {
                                        "Data Fluxo": [
                                                        dt.date(2027, 1, 1),
                                                        dt.date(2026, 7, 1),
                                                        dt.date(2026, 1, 1),
                                                        dt.date(2025, 7, 1),
                                                        dt.date(2025, 1, 1),
                                                        dt.date(2024, 7, 1),
                                                        dt.date(2024, 1, 1),
                                                        dt.date(2023, 7, 1),
                                                        dt.date(2023, 1, 1),
                                                        dt.date(2022, 7, 1),
                                                    ],
                                        "Juros": [48.8088] * 10,
                                        "Amortização": [1000] + [0] * 9,
                                        "VF": [1048.8088] + [48.8088] * 9,
                                        "Prazo DU": [1146, 1018, 896, 765, 643, 513, 389, 264, 140, 13],
                                        "FD": [
                                                1.718912511,
                                                1.617996618,
                                                1.527330912,
                                                1.435625391,
                                                1.355178999,
                                                1.274412243,
                                                1.201862824,
                                                1.132907851,
                                                1.068413959,
                                                1.006163765,
                                            ],
                                        "PU": [610.16, 30.17, 31.96, 34.00, 36.02, 38.30, 40.61, 43.08, 45.68, 48.51],
                                    }
                                )

# Ajusta tipos do resultado_previsto
resultado_previsto["Data Fluxo"] = resultado_previsto["Data Fluxo"].astype("datetime64[ns]")

dt_ini = dt.date(2022, 6, 10)
dt_fim = dt.date(2027, 1, 1)
tir = 0.1265
resultado = calcula_pu_ntnf(dt_fim, dt_ini, tir)
fazTeste(resultado_previsto, resultado)

Teste sucedido!


True